In [ ]:
! pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=bef4d00a326e8e7f9c129ac61b3b445d43cde46ac29d763ade69f4c851a30eb0
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
#Import the dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

#Create lists to store the scraped data
authors = []
dates = []
statements = []
sources = []
targets = []
raw_targets= []

#Create a function to scrape the site
def scrape_website(page_number, source):
    page_num = str(page_number) #Convert the page number to a string


    '''source: all'''
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num #append the page number to complete the URL

    webpage = requests.get(URL)  #Make a request to the website
    #time.sleep(3)
    soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
    #Get the tags and it's class
    statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
    target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class

    #loop through the footer class m-statement__footer to get the date and author
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)
    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        try:
            if detect(statement_text) == 'en':
                statements.append(statement_text)
            else:
                statements.append("not english")
        except:
            continue  # Skip if language detection fails
    #Loop through the div m-statement__meta to get the source
    for i in statement_meta:
        link3 = i.find_all('a') #Source
        source_text = link3[0].text.strip()
        sources.append(source_text)
    #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
    counter = 0
    for index, each in enumerate(target):
        counter += 1
        fact = each.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        raw_targets.append(fact)
        if 'true' in fact and 'barely-true' not in fact :
          targets.append(1)
        elif 'flip' in fact or 'flop' in fact or 'not english' in statements[index]:
          targets.append(2)
        else:
          targets.append(0)



#Loop through 'n-1' webpages to scrape the data
n=100
for i in range(1, n):
    scrape_website(i, source='')
print(len(authors))
print(len(dates))
print(len(statements))
print(len(sources))
print(len(targets))
#Create a new dataFrame
data = pd.DataFrame(columns = ['title',  'label'])
# data['author'] = authors
data['title'] = statements
# data['source'] = sources
# data['date'] = dates
data['label'] = targets

data = data[data['label'] != 2]
data = data.reset_index(drop=True)

# data['raw_target'] = raw_targets
#Show the data set
print(data)
print(set(raw_targets))

print(data['label'].value_counts())
data.to_csv('politifact-sample.csv', index=False, sep=',')

2970
2970
2970
2970
2970
                                                  title  label
0     Arizona Democrats “needed a lawsuit to force t...      0
1     Former U.S. Rep. Liz Cheney wrote a viral lett...      0
2     Under the SAVE Act, “as long as you have a Rea...      0
3     “Less than half of Americans have $1,000 in sa...      0
4     Cody Balmer, the person charged with setting f...      0
...                                                 ...    ...
2772  Roseanne Barr made a video of a rainbow in a s...      0
2773         “Arizona BANS Electronic Voting Machines.”      0
2774                                        not english      0
2775  “Miami-Dade County has banned” Amanda Gorman’s...      0
2776  A chart on Arctic sea ice provides evidence th...      1

[2777 rows x 2 columns]
{'false', 'half-flip', 'half-true', 'true', 'pants-fire', 'mostly-true', 'barely-true', 'full-flop'}
label
0    2512
1     265
Name: count, dtype: int64
